In [1]:
# TESTING THE CODE

import requests
from pdbfixer import PDBFixer
from openmm.app import *
from openmm import *
from openmm.unit import *
import mdtraj as md

# Step 1: Download the PDB file for 1VII
pdb_id = "1VII"
pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
pdb_filename = f"{pdb_id}.pdb"

response = requests.get(pdb_url)
with open(pdb_filename, 'wb') as file:
    file.write(response.content)

# Step 2: Preprocess the PDB file using PDBFixer
fixer = PDBFixer(filename=pdb_filename)
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens()

# Use the AMBER force field
forcefield = ForceField('amber14-all.xml', 'amber14/tip3p.xml')

# Step 3: Set up the simulation
modeller = Modeller(fixer.topology, fixer.positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometer)

system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, 
                                 nonbondedCutoff=1.0*nanometer, constraints=HBonds)

integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

# Minimize energy
simulation.minimizeEnergy()

# Equilibrate
simulation.context.setVelocitiesToTemperature(300*kelvin)
simulation.step(10000)  # 20 ps equilibration

# Step 4: Run the simulation for 1 ns
simulation.reporters.append(DCDReporter('trajectory.dcd', 1000))
simulation.reporters.append(StateDataReporter('output.log', 1000, step=True, 
                                              potentialEnergy=True, temperature=True))

simulation.step(500000)  # 1 ns simulation

# Step 5: Analyze the secondary structure
traj = md.load('trajectory.dcd', top=md.Topology.from_openmm(modeller.topology))

# Get secondary structure at the beginning and end
initial_structure = md.compute_dssp(traj[0])
final_structure = md.compute_dssp(traj[-1])

# Compare secondary structures
initial_counts = {ss: list(initial_structure[0]).count(ss) for ss in set(initial_structure[0])}
final_counts = {ss: list(final_structure[0]).count(ss) for ss in set(final_structure[0])}

print("Initial Secondary Structure Counts:", initial_counts)
print("Final Secondary Structure Counts:", final_counts)

# Check for changes
changes = {ss: final_counts.get(ss, 0) - initial_counts.get(ss, 0) for ss in set(initial_counts) | set(final_counts)}
print("Changes in Secondary Structure:", changes)

Initial Secondary Structure Counts: {'NA': 2666, 'H': 17, 'C': 19}
Final Secondary Structure Counts: {'NA': 2666, 'H': 19, 'C': 17}
Changes in Secondary Structure: {'NA': 0, 'H': 2, 'C': -2}


In [1]:
!ls

1VII.pdb  exp_5.ipynb  exp_5_test_code.ipynb  output.log  trajectory.dcd


In [5]:
import mdtraj as md

traj = md.load('1VII.pdb')
print("Number of residues in total: ",traj.n_residues)
print("Number of chains: ",traj.n_chains)

secondary_structure = md.compute_dssp(traj,simplified=True)[0]
print("Number of residues in sheets: ",len([i for i in secondary_structure if i == 'E']))
print("Number of residues in helices: ",len([i for i in secondary_structure if i == 'H']))
print("Number of residues in coils: ",len([i for i in secondary_structure if i == 'C']))

Number of residues in total:  36
Number of chains:  1
Number of residues in sheets:  0
Number of residues in helices:  19
Number of residues in coils:  17
